### 	
 Trabalho Final de ML

Os alunos devem apresentar um modelo de Machine Learning elaborado para os dados da empresa Orange da copa KDD de 2009. Os dados e a documentação sobre o dataset estão no linkwww.vincentlemaire-labs.fr/kddcup2009/.

Vocês devem trabalhar com os dados SMALL. Escolham uma das três opção de variáveis target (Churn, Appetency ou Upselling) e montem um modelo de ML que preveja o outcome escolhido.

O modelo deve ser entregue num Jupyter Notebook ou num RMarkdown com os códigos e resultados COMENTADOS do exercício. Você deve mostrar o que foi feito, porque escolheu o modelo e qual tratamento foi dado aos dados e por quê.

O exerício será entregue pelo Github através deste link: classroom.github.com/g/jnKXDsi7

DATA PREPARATION
trabalhando com daods desbalanceados
Vamos utilizar o algoritmo SMOTE( Sintetic minority Over-Sampling Technique).
Primeiro, vamos fazer o dowload dos dados que precisamos para o exercicio:

In [0]:
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train.data.zip
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_churn.labels
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_appetency.labels
!wget http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_upselling.labels

--2019-06-26 00:16:40--  http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train.data.zip
Resolving www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)... 81.169.145.72, 2a01:238:20a:202:1072::
Connecting to www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)|81.169.145.72|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8424904 (8.0M) [application/zip]
Saving to: ‘orange_small_train.data.zip’

orange_small_train. 100%[===================>]   8.03M   630KB/s    in 13s     

2019-06-26 00:16:54 (610 KB/s) - ‘orange_small_train.data.zip’ saved [8424904/8424904]

--2019-06-26 00:16:55--  http://www.vincentlemaire-labs.fr/kddcup2009/orange_small_train_churn.labels
Resolving www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)... 81.169.145.72, 2a01:238:20a:202:1072::
Connecting to www.vincentlemaire-labs.fr (www.vincentlemaire-labs.fr)|81.169.145.72|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196328 (192K)
Saving to: ‘

In [0]:
!unzip orange_small_train.data.zip

Archive:  orange_small_train.data.zip
  inflating: orange_small_train.data  
 extracting: orange_small_train.data.cksum  


In [0]:
!ls

gdrive				     orange_small_train.data.cksum
orange_small_train_appetency.labels  orange_small_train.data.zip
orange_small_train_churn.labels      orange_small_train_upselling.labels
orange_small_train.data		     sample_data


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,KFold
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import  RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score,confusion_matrix


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
X = pd.read_csv("orange_small_train.data", sep="\t")

In [0]:
X.shape

(50000, 230)

In [0]:
Y = pd.read_csv("orange_small_train_upselling.labels", sep="\t", header=None)

In [0]:
Y.shape

(50000, 1)

In [0]:
X = pd.read_csv("orange_small_train.data", sep="\t")
y = pd.read_csv("orange_small_train_appetency.labels", header=None)
print(X.shape)
print(y.shape)

(50000, 230)
(50000, 1)


Vamos criar uma regra em que variáveis com mais de 15% de missing serão cortadas do banco.

In [0]:
X.shape

(50000, 230)

In [0]:
colunas_dropar = []
for i in range(X.shape[1]):
  if X.iloc[:,i].isna().sum() / X.shape[0] > 0.15:
    colunas_dropar.append(X.columns[i])
    
X = X.drop(columns = colunas_dropar)

X.shape

(50000, 66)

Começar o processo de pré-processamento

In [0]:
medimp = SimpleImputer(missing_values=np.nan, strategy='mean')
X.iloc[:, :38] = medimp.fit_transform(X.iloc[:, :38])
X.head()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,Var44,Var57,Var65,Var73,Var74,Var76,Var78,Var81,Var83,Var85,Var109,Var112,Var113,Var119,Var123,Var125,Var132,Var133,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var173,Var181,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
0,1526.000000,7.0,184.0,464.000000,580.0,14.000000,128.0,166.56,0.0,3570.0,0.0,4.076907,9.0,36.0,35.0,1350864.0,0.0,7333.110000,5.0,12.0,104.00000,168.0,117625.60,1175.000000,6.0,720.0,0.0,1212385.0,69134.0,185.0,0.0,9.000000,397579.000000,1812252.0,142.0,38418.0,0.0,0.0,bZkvyxLkBI,RO12,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,uKAI,L84s,XfqtO3UdzaXh_,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,xb3V,RAYp,F2FyR07IdsN7I
1,525.000000,0.0,0.0,168.000000,210.0,2.000000,24.0,353.52,0.0,4764966.0,0.0,5.408032,9.0,26.0,0.0,2872928.0,3.0,151098.900000,25.0,2.0,40.00000,40.0,-356411.60,590.000000,72.0,0.0,8.0,4136430.0,357038.0,0.0,0.0,9.000000,278334.000000,10439160.0,32.0,238572.0,0.0,0.0,CEat0G8rTN,RO12,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,kZJyVg2,NaN,NaN,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,fKCe,RAYp,F2FyR07IdsN7I
2,5236.000000,7.0,904.0,1212.000000,1515.0,26.000000,816.0,220.08,0.0,5883894.0,0.0,6.599658,9.0,130.0,518.0,1675776.0,0.0,16211.580000,40.0,58.0,312.00000,336.0,405104.00,3230.000000,114.0,5967.0,0.0,3478905.0,248932.0,800.0,0.0,36.000000,320565.000000,9826360.0,206.0,434946.0,0.0,0.0,eOQt0GoOh3,AERks4l,taul,1K8T,ffXs,NldASpP,y4g9XoZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,uKAI,L84s,UbxQ8lZ,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,Qu4f,02N6s8f,ib5G6X1eUxUn6
3,1326.437116,0.0,0.0,234.518225,0.0,4.507926,0.0,22.08,0.0,0.0,0.0,1.988250,9.0,12.0,0.0,0.0,0.0,103084.052693,0.0,0.0,60.88866,0.0,-275703.60,916.112185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.727665,294920.804255,0.0,0.0,0.0,0.0,0.0,jg69tYsGvO,RO12,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,W8mQ,9_Y1,YULl,VpdQ,NaN,me75fM6ugJ,kIsH,uKAI,Mtgm,NhsEn4L,kq0dQfu,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
4,1029.000000,7.0,3216.0,64.000000,80.0,4.000000,64.0,200.00,0.0,0.0,0.0,4.552446,18.0,82.0,224.0,784448.0,0.0,37423.500000,0.0,0.0,32.00000,56.0,10714.84,215.000000,0.0,15111.0,0.0,150650.0,66046.0,3255.0,0.0,9.000000,267162.000000,644836.0,2.0,0.0,0.0,0.0,IXSgUHShse,RO12,taul,1K8T,uNkU,EKR938I,ThrHXVS,xklU,9_Y1,RVjC,sJzTlal,6JmL,me75fM6ugJ,kIsH,uKAI,L84s,XfqtO3UdzaXh_,11p4mKe,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I


In [0]:
## Nas categóricas vamos substituir os missing por uma categoria chamada Miss
missimp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = "Miss")
X.iloc[:, 38:68] = missimp.fit_transform(X.iloc[:, 38:68])

Antes de transformar as categóricas em dummies, retirar as categorias raras.

In [0]:
X = pd.get_dummies(X)
X.columns

Index(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25', 'Var28',
       'Var35', 'Var38',
       ...
       'Var228__URKn_77G3QyQHSVQ2N1RKmtW', 'Var228__cTCyH95OE93jSkoIBT',
       'Var228_am14IcfM7tWLrUmRT52KtA', 'Var228_b9qbUNk0dML_Mvi2',
       'Var228_d0LtHjWeaXyArdN4sxU_saXqH', 'Var228_ib5G6X1eUxUn6',
       'Var228_iyHGyLCEkQ', 'Var228_n1OBWGkV3fbsHR75taC', 'Var228_r_7E',
       'Var228_xwM2aC7IdeMC0'],
      dtype='object', length=40708)